In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import konlpy
from konlpy.tag import Okt
from konlpy.tag import Kkma
import numpy as np
okt = Okt()
kkma = Kkma()

In [75]:
import pandas as pd

try:
    with open('../김나윤/뉴스/셀트리온_20210401-20230701.csv', encoding='ANSI') as file:
        news_data = pd.read_csv(file)
except UnicodeDecodeError:
    with open('../김나윤/뉴스/셀트리온_20210401-20230701.csv', encoding='utf-8', errors='replace') as file:
        news_data = pd.read_csv(file)

# Cleaning

In [76]:
news_data = news_data[['키워드']]

In [77]:
news_data['키워드'] = news_data['키워드'].replace(',', ' ', regex=True)

In [15]:
# import re
# regex = r'[^\w\s]'
# # text 열의 모든 값을 대상으로 정규표현식을 적용하여 특수문자를 제거
# news_data['본문'] = news_data['본문'].apply(lambda x: re.sub(regex, '', str(x)))
# news_data['제목'] = news_data['제목'].apply(lambda x: re.sub(regex, '', str(x)))

In [53]:
#!pip install konlpy
from konlpy.tag import Kkma        
kkma = Kkma()
from konlpy.tag import Hannanum    
hannanum = Hannanum()
from konlpy.tag import Okt         
t = Okt() 
from konlpy.tag import *
import nltk
import pickle
import copy
from tqdm import tqdm

In [54]:
pos_tag = []

# news_data 데이터프레임의 각 행에 대해 반복
for _, row in tqdm(news_data.iterrows()):
    # 현재 행의 content 값을 news_text 변수에 할당
    news_text = row['키워드']
    # PosTagging
    tokens_ko = t.pos(news_text)
    pos_tag.append(tokens_ko)

8390it [03:36, 38.76it/s]


In [10]:
#첫번째 행의 Pos Tagging
print(pos_tag[0])

[('확대', 'Noun'), ('제품', 'Noun'), ('영역', 'Noun'), ('셀트리온', 'Noun'), ('시', 'Modifier'), ('밀러', 'Noun'), ('안과', 'Noun'), ('질환', 'Noun'), ('치료', 'Noun'), ('제', 'Noun'), ('시', 'Modifier'), ('밀러', 'Noun'), ('품목', 'Noun'), ('허가', 'Noun'), ('신청', 'Noun'), ('CT', 'Alpha'), ('-', 'Punctuation'), ('P', 'Alpha'), ('아', 'Exclamation'), ('일', 'Modifier'), ('리아', 'Noun'), ('바이오', 'Noun'), ('밀러', 'Noun'), ('CT', 'Alpha'), ('-', 'Punctuation'), ('P', 'Alpha'), ('품목', 'Noun'), ('허가', 'Noun'), ('신청', 'Noun'), ('추진', 'Noun'), ('현지', 'Noun'), ('성인', 'Noun'), ('적응', 'Noun'), ('증', 'Noun'), ('품목', 'Noun'), ('허가', 'Noun'), ('매출', 'Noun'), ('아', 'Exclamation'), ('일', 'Modifier'), ('리아', 'Noun'), ('글로벌', 'Noun'), ('12조', 'Number'), ('자가면역질환', 'Noun'), ('항암제', 'Noun'), ('확대', 'Noun'), ('안과', 'Noun'), ('질환', 'Noun'), ('영역', 'Noun'), ('셀트리온', 'Noun'), ('29일', 'Number'), ('현지', 'Noun'), ('시간', 'Noun'), ('아', 'Exclamation'), ('일', 'Modifier'), ('리아', 'Noun'), ('안과', 'Noun'), ('질환', 'Noun'), ('치료', 'Noun'), ('제', 'No

In [57]:
normalization_li = []
for pos in pos_tag: 
    in_li = []
    for ele in pos:
        #품사가 조사, 접속사이면 continue
        if ele[1] in ['Josa','Suffix']:
            continue
        in_li.append(ele[0])
    normalization_li.append(in_li)
print(normalization_li[0])

['확대', ',', '제품', ',', '영역', ',', '셀트리온', ',', '시', '밀러', ',', '안과', ',', '질환', ',', '치료', '제', ',', '시', '밀러', ',', '품목', ',', '허가', ',', '신청', ',', 'CT', '-', 'P', ',', '아', '일', '리아', ',', '바이오', ',', '밀러', ',', 'CT', '-', 'P', ',', '품목', ',', '허가', ',', '신청', ',', '추진', ',', '현지', ',', '성인', ',', '적응', '증', ',', '품목', ',', '허가', ',', '매출', ',', '아', '일', '리아', ',', '글로벌', ',', '12조', ',', '자가면역질환', ',', '항암제', ',', '확대', ',', '안과', ',', '질환', ',', '영역', ',', '셀트리온', ',', '29일', ',', '현지', '시간', ',', '아', '일', '리아', ',', '안과', ',', '질환', ',', '치료', '제', ',', 'EYLEA', ',', '애플', '리버', '셉트', ',', '바이오', ',', '밀러', ',', 'CT', '-', 'P', ',', '품목', '허가', ',', '미국', ',', '식품', ',', '의', '약국', ',', 'FDA', ',', '신청', ',', '30일', ',', 'CT', '-', 'P', ',', '셀트리온', ',', '제품', ',', '포트폴리오', ',', '확장', ',', '바이오', '시', '밀러', ',', '오', '리지', '널', ',', '제품', ',', '아', '일', '리아', ',', '12조', ',', '규모', ',', '글로벌', ',', '시장', ',', '형성', ',', '독점', ',', '아', '일', '리아', ',', '미국', ',', '독점', ',', '내년'

In [26]:
with open('../김나윤/data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords] # stopword 파일의 줄바꿈 문자 제거
okt = Okt()

In [59]:
tokens = normalization_li
token_stop = []
for token in tokens:
    in_li = []
    for tok in token:
        if tok not in stopwords:
            in_li.append(tok)
    token_stop.append(in_li)
print(token_stop[0]) 

['확대', ',', '제품', ',', '영역', ',', '셀트리온', ',', '시', '밀러', ',', '안과', ',', '질환', ',', '치료', ',', '시', '밀러', ',', '품목', ',', '허가', ',', '신청', ',', 'CT', '-', 'P', ',', '리아', ',', '바이오', ',', '밀러', ',', 'CT', '-', 'P', ',', '품목', ',', '허가', ',', '신청', ',', '추진', ',', '현지', ',', '성인', ',', '적응', '증', ',', '품목', ',', '허가', ',', '매출', ',', '리아', ',', '글로벌', ',', '12조', ',', '자가면역질환', ',', '항암제', ',', '확대', ',', '안과', ',', '질환', ',', '영역', ',', '셀트리온', ',', '29일', ',', '현지', ',', '리아', ',', '안과', ',', '질환', ',', '치료', ',', 'EYLEA', ',', '애플', '리버', '셉트', ',', '바이오', ',', '밀러', ',', 'CT', '-', 'P', ',', '품목', '허가', ',', '미국', ',', '식품', ',', '약국', ',', 'FDA', ',', '신청', ',', '30일', ',', 'CT', '-', 'P', ',', '셀트리온', ',', '제품', ',', '포트폴리오', ',', '확장', ',', '바이오', '시', '밀러', ',', '리지', '널', ',', '제품', ',', '리아', ',', '12조', ',', '규모', ',', '글로벌', ',', '시장', ',', '형성', ',', '독점', ',', '리아', ',', '미국', ',', '독점', ',', '내년', ',', '유럽', ',', '물질', ',', '특허', ',', '만료', ',', '예정', ',', '특허', ',', '종료

In [78]:
import scipy as sp
import pandas as pd
import numpy as np

from konlpy.tag import Kkma        ; kkma = Kkma()
from konlpy.tag import Hannanum    ; hannanum = Hannanum()
from konlpy.tag import Okt         ; t = Okt()     # 구 트위터
from konlpy.tag import *
import nltk
import pickle

from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
#import graphviz
from sklearn.tree import export_graphviz

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [80]:
preprocess = news_data

In [82]:
news_text = ''
news_data = preprocess.reset_index(drop=True)
for _, row in news_data.iterrows():
    news_text += row['키워드']

In [83]:
tokens_ko = t.morphs(news_text)
ko = nltk.Text(tokens_ko)

print(len(ko.tokens))          # 토큰 전체 개수
print(len(set(ko.tokens)))     # 토큰 unique 개수
print(ko.vocab().most_common(20))

2181069
46249
[('셀트리온', 34141), ('바이오', 32515), ('기업', 16563), ('시장', 14600), ('하락', 12758), ('상승', 12366), ('거래', 12149), ('제', 11709), ('코스피', 11663), ('미국', 11595), ('투자', 10706), ('적', 10511), ('지수', 9996), ('종목', 9966), ('이', 9873), ('주가', 9524), ('포인트', 9476), ('들', 9239), ('치료', 8971), ('외국인', 8916)]


In [84]:
stop_words = ['바이오','삼성','성','삼','로', '적', '이', '셀트리온']

In [62]:
tokens_ko = [each_word for each_word in tokens_ko
           if each_word not in stop_words]


ko = nltk.Text(tokens_ko)

# 그래프에서 한글 폰트가 깨질 경우 실행
# from matplotlib import font_manager, rc
# font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
# rc('font', family=font_name)

# plt.figure(figsize=(15,6))
# ko.plot(50)
# plt.show()

In [63]:
num

8390

In [87]:
data = ko.vocab().most_common(500000)

In [53]:
df['단어'] = [re.sub('[^A-Za-z0-9가-힣]', '', s) for s in df['단어']]

In [89]:
df = pd.DataFrame(data, columns=['단어','빈도'])

In [19]:
import pandas as pd

df = pd.read_csv('./셀트리온_단어.csv', index_col=0)
check = df[(df['빈도'] == 0) | (df['빈도'] == -1)]
print('\n', '한거 : ' , len(check))
check


 한거 :  0


,단어,빈도


In [5]:
check = pd.DataFrame(check)

AttributeError: 'list' object has no attribute 'info'

In [8]:
check.to_csv('긍부정_단어(156).csv', index= 0)

In [39]:
len(data)

141

In [34]:
data = ko.vocab().most_common(num)